In [1]:
import pandas as pd
import numpy as np

df = pd.read_parquet('baseline_with_lags.parquet')

In [2]:
df['foto_mes'] = pd.to_datetime(df['foto_mes'], format='%Y%m')
# df['target'] = df['target'].map({'C': 0, 'B2': 1})
# df['target'].value_counts()

In [3]:
df_val = df[df['foto_mes']=='2021-07-01']
df_train = df[df['foto_mes']<'2021-07-01']

In [4]:
# just a list of clients to recover true label, B1 or B2
b1_clients = pd.read_csv('B1_clients_202107.csv')
b1 = set(b1_clients['numero_de_cliente'])
df_val['target'] = np.where(df_val['numero_de_cliente'].isin(b1), 'B1', df_val['target'])

/tmp/ipykernel_2358886/3571757368.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['target'] = np.where(df_val['numero_de_cliente'].isin(b1), 'B1', df_val['target'])


In [5]:
df_val.shape, df_train.shape

((164682, 374), (1460661, 374))

In [6]:
# X_train = df_train.drop(columns=['target'])
# X_val = df_val.drop(columns=['target'])
# y_train = df_train['target']
# y_val = df_val['target']

In [7]:
X_val = df_val.drop(columns=['target'])
y_val = df_val['target']

In [8]:
%%time
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(df_train)
val_data = TabularDataset(df_val)

CPU times: user 1.48 s, sys: 3.61 s, total: 5.09 s
Wall time: 873 ms


/home/ynesterov/autogluon/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from autogluon.core.metrics import make_scorer

def normalized_ganancia(y_true: np.array,
                      y_pred: np.array) -> float:
    # Floats to avoid underflow
    ideal_value = np.sum(y_true)*0.273
    ganancia = y_true*0.280 - 0.007
    ganancia_df = pd.DataFrame({'prob': y_pred, 'ganancia': ganancia})
    ganancia_df = ganancia_df.sort_values(by='prob', ascending=False).reset_index(drop=True)
    ganancia_df['ganancia_roll'] = ganancia_df['ganancia'].cumsum()
    
    gan_max = ganancia_df['ganancia_roll'].max()
    ind = ganancia_df[ganancia_df['ganancia_roll']==gan_max].index[0]
    lo = ind-2000 if ind>2000 else 0
    res = np.mean(ganancia_df['ganancia_roll'].iloc[lo:ind])
    
    normalized_res = res/ideal_value
    print(f'ideal_value {ideal_value} res {res}, ind {ind}, gan_max {gan_max} normalized_res {normalized_res}')
    return normalized_res

ag_ganancia_custom_scorer = make_scorer(name='normalized_ganancia',
                                      score_func=normalized_ganancia,
                                      optimum=1,
                                      greater_is_better=True)

In [9]:
predictor = TabularPredictor(label='target', eval_metric=ag_ganancia_custom_scorer).fit(
    train_data,
    time_limit=3600,
    num_cpus=24,
    excluded_model_types = ['KNN', 'XT' ,'RF'],
    num_stack_levels=0,
    fit_weighted_ensemble=False,
)

No path specified. Models will be saved in: "AutogluonModels/ag-20231120_210737/"
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20231120_210737/"
AutoGluon Version:  0.8.2
Python Version:     3.9.7
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #85-Ubuntu SMP Fri Jul 7 15:25:09 UTC 2023
Disk Space Avail:   395.60 GB / 982.70 GB (40.3%)
Train Data Rows:    1460661
Train Data Columns: 373
Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerat

ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan

	0.0236	 = Validation score   (normalized_ganancia)
	33.27s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 3515.61s of the 3515.61s of remaining time.


ideal_value 40.677 res 0.8295000195503235, ind 10, gan_max 1.6030000448226929 normalized_res 0.02039235979915735
ideal_value 40.677 res 0.9605555555555557, ind 9, gan_max 1.8900000000000001 normalized_res 0.023614218245090732
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.23800000548362732 normalized_res -0.000516262300059569
ideal_value 40.677 res -0.021000001579523087, ind 5, gan_max 0.238

	0.0273	 = Validation score   (normalized_ganancia)
	35.03s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 3480.44s of the 3480.44s of remaining time.


...

Fitting model: XGBoost ... Training model for up to 207.97s of the 207.96s of remaining time.


ideal_value 40.677 res 0.6825000047683716, ind 4, gan_max 1.3650000095367432 normalized_res 0.01677852360715814
ideal_value 40.677 res 0.45266667008399963, ind 3, gan_max 0.8119999766349792 normalized_res 0.011128319937163499
ideal_value 40.677 res 0.45266667008399963, ind 3, gan_max 0.8119999766349792 normalized_res 0.011128319937163499
ideal_value 40.677 res 0.45266667008399963, ind 3, gan_max 0.8119999766349792 normalized_res 0.011128319937163499
ideal_value 40.677 res 0.45266667008399963, ind 3, gan_max 0.8119999766349792 normalized_res 0.011128319937163499
ideal_value 40.677 res 0.45266667008399963, ind 3, gan_max 0.8119999766349792 normalized_res 0.011128319937163499
ideal_value 40.677 res 0.45266667008399963, ind 3, gan_max 0.8119999766349792 normalized_res 0.011128319937163499
ideal_value 40.677 res 0.2695000171661377, ind 2, gan_max 0.5390000343322754 normalized_res 0.006625366107779278
ideal_value 40.677 res 0.27300000190734863, ind 1, gan_max 0.5460000038146973 normalized_re

	0.0168	 = Validation score   (normalized_ganancia)
	71.95s	 = Training   runtime
	0.52s	 = Validation runtime
Fitting model: NeuralNetTorch ... Training model for up to 135.45s of the 135.44s of remaining time.


ideal_value 40.677 res 0.6825000000000001, ind 4, gan_max 1.3650000000000002 normalized_res 0.01677852348993289


	Time limit exceeded... Skipping NeuralNetTorch.
AutoGluon training complete, total runtime = 3670.43s ... Best model: "CatBoost"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20231120_210737/")


In [11]:
predictor = TabularPredictor.load("AutogluonModels/ag-20231120_210737/")
preds = predictor.predict_proba(X_val)
y_val = np.where(y_val=='B2', 1, 0)

In [13]:
def ganancia_integral(probs: np.array, 
                      y_true: np.array) -> float:
    ganancia_df = pd.DataFrame({'prob': probs, 'truth': y_true})
    ganancia_df = ganancia_df.sort_values(by='prob', ascending=False).reset_index(drop=True)
    ganancia_df['ganancia_ind'] = np.where(ganancia_df['truth']==1, 273, -7)
    ganancia_df['ganancia_roll'] = ganancia_df['ganancia_ind'].cumsum()
    gan_max = ganancia_df['ganancia_roll'].max()
    ind = ganancia_df[ganancia_df['ganancia_roll']==gan_max].index[0]
    print(ind)
    res = np.mean(ganancia_df['ganancia_roll'].iloc[ind-1000:ind+1001])
    return res

In [13]:
# With B2+B1 target engineering
ganancia_integral(preds[1], y_val)

17906


408816.27736131934

In [14]:
# True value
ganancia_integral(preds[1], y_val)

13823


143097.26736631684